### 1.Importing Libraries

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

from geopy.geocoders import Nominatim 

from pandas.io.json import json_normalize
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### 2.Getting data from a website providing the areas and their postal code

In [2]:
url = 'http://zipcodepincode.com/India/Maharashtra/Pune/Pune-City/index.html'

In [3]:
Pune_data, = pd.read_html(url,match = 'Pin Code')
Pune_data.head()

,Post Office,Pin Code
0,9 DRD,411014
1,A RShala,411004
2,AFMC,411040
3,Airport (Pune),411032
4,Akurdi,411035


In [4]:
Pune_data = Pune_data[['Pin Code','Post Office']]

In [5]:
Pune_data.head()

,Pin Code,Post Office
0,411014,9 DRD
1,411004,A RShala
2,411040,AFMC
3,411032,Airport (Pune)
4,411035,Akurdi


In [6]:
Pune_data['Pin Code'].value_counts()

411001    8
411002    6
411030    6
411003    5
411042    5
411037    4
411033    4
411048    4
411038    3
411032    3
411018    3
411017    3
411016    3
411015    3
411014    3
411011    3
411004    3
411005    3
411012    2
411019    2
411020    2
411026    2
411028    2
411009    2
411007    2
411036    2
411008    2
411006    2
411062    2
411040    2
411052    2
411044    2
411045    1
411060    1
411058    1
411013    1
411057    1
411051    1
411047    1
411046    1
411035    1
411034    1
411022    1
411023    1
411024    1
411043    1
411027    1
411041    1
411031    1
411039    1
411021    1
Name: Pin Code, dtype: int64

In [7]:
Pune_data.sort_values('Pin Code',ascending = True,inplace= True)

In [8]:
Pune_data.reset_index(drop=True, inplace=True)

In [9]:
Pune_data.shape

(119, 2)

### 3.Getting loction data

In [10]:
Latitudes = []
Longitudes = []
length = len(Pune_data['Post Office'])
for i in range(0,length):
    address = '{},pune'.format(Pune_data['Post Office'][i])
    geolocator = Nominatim(user_agent="my-application",timeout=3)
    location = geolocator.geocode(address)
    if location == None:
        latitude = 'None'
        longitude = 'None'
    else:
        latitude = location.latitude
        longitude = location.longitude
    Latitudes.append(latitude)
    Longitudes.append(longitude)




In [11]:
Pune_data['Latitudes'] = Latitudes
Pune_data['Longitudes'] = Longitudes
new_Pune_data = Pune_data[Pune_data['Latitudes'] != 'None']
new_Pune_data.reset_index(drop=True,inplace=True)
new_Pune_data.shape

(68, 4)

In [12]:
new_Pune_data.head()

,Pin Code,Post Office,Latitudes,Longitudes
0,411001,Pune,18.5214,73.8545
1,411001,Sachapir Street,18.5151,73.8762
2,411002,Nana Peth,18.5184,73.8563
3,411002,Pune City,18.5214,73.8545
4,411002,Shukrawar Peth (Pune),18.5114,73.854


In [13]:
address = 'Pune'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Pune are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Pune are 18.521428, 73.8544541.


In [14]:
map_pune = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, Post_office in zip(new_Pune_data['Latitudes'], new_Pune_data['Longitudes'], new_Pune_data['Post Office']):
    label = '{}'.format(Post_office)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_pune)  
    
map_pune

In [15]:
wrong_loc = ['Sangavi','Kalewadi','Pimpri P F','Range Hills','Salisbury Park']
for i in range(0,5):
      new_Pune_data = new_Pune_data[new_Pune_data['Post Office'] != '{}'.format(wrong_loc[i])]

In [16]:
new_Pune_data.reset_index(drop=True,inplace = True)

In [17]:
map_pune = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, Post_office in zip(new_Pune_data['Latitudes'], new_Pune_data['Longitudes'], new_Pune_data['Post Office']):
    label = '{}'.format(Post_office)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_pune)  
    
map_pune

### 4.Defining Foursquare credentials

In [18]:
CLIENT_ID = 'T1K4TNUNOFCIGVQ3VUEF1ZRDQRKICFROYMTABJT1TR2MDZ2K' # your Foursquare ID
CLIENT_SECRET = 'E5NBWIC0214SCUKSZSLZODMYV1TPLJEXC3BFQ4C1LVJANLC1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: T1K4TNUNOFCIGVQ3VUEF1ZRDQRKICFROYMTABJT1TR2MDZ2K
CLIENT_SECRET:E5NBWIC0214SCUKSZSLZODMYV1TPLJEXC3BFQ4C1LVJANLC1


### 5.Exploring Neighborhoods

In [19]:
neighborhood_latitude = new_Pune_data.loc[0, 'Latitudes'] # neighborhood latitude value
neighborhood_longitude = new_Pune_data.loc[0, 'Longitudes'] # neighborhood longitude value

neighborhood_name = new_Pune_data.loc[0, 'Post Office'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Pune are 18.521428, 73.8544541.


In [20]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=T1K4TNUNOFCIGVQ3VUEF1ZRDQRKICFROYMTABJT1TR2MDZ2K&client_secret=E5NBWIC0214SCUKSZSLZODMYV1TPLJEXC3BFQ4C1LVJANLC1&v=20180605&ll=18.521428,73.8544541&radius=500&limit=100'

In [21]:
import requests
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ef807adf9762512a1e113e6'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Pune',
  'headerFullLocation': 'Pune',
  'headerLocationGranularity': 'city',
  'totalResults': 7,
  'suggestedBounds': {'ne': {'lat': 18.525928004500006,
    'lng': 73.8591910533527},
   'sw': {'lat': 18.516927995499994, 'lng': 73.8497171466473}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4f579db56b740547b24e5d3a',
       'name': 'Lal Mahal',
       'location': {'address': 'Corner of Kasba Ganpati Mandir,',
        'crossStreet': 'off Shivaji Road,',
        'lat': 18.518719674058865,
        'lng': 73.85655641555786,
        'labeledLatLngs': [{'label': 'display'

In [22]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\Ninad\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Lal Mahal,Historic Site,18.518720,73.856556
1,Shaniwar Wada,Historic Site,18.520151,73.855187
2,Sai Service Bajaj,Motorcycle Shop,18.522260,73.850185
3,New Poona Bakery,Bakery,18.517028,73.854845
4,New Vaijanath Amrutatulya,Snack Place,18.519178,73.851471


In [24]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

7 venues were returned by Foursquare.


In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
new_Pune_data_venues = getNearbyVenues(names=new_Pune_data['Post Office'],
                                   latitudes=new_Pune_data['Latitudes'],
                                   longitudes=new_Pune_data['Longitudes']
                                  )


Pune
Sachapir Street
Nana Peth
Pune City
Shukrawar Peth (Pune)
Bajirao Road
East Khadki
Khadki
Khadki Bazar
Film Institute
Deccan Gymkhana
Shivajinagar (Pune)
Yerwada
Aundh T S
Ganeshkhind
Baner Road
Parvati
Kasba Peth
Rasta Peth
Mangalwar Peth (Pune)
Dapodi
Vishrantwadi
Dhanori
Shivaji Housing Society
Model Colony
Nehrunagar (Pune)
Chinchwad East
M Phulenagar
Botanical Garden (Pune)
Armament
Srpf
Khadakwasla R S
Bhosari I E
Hadapsar
Shaniwar Peth (Pune)
Sadashiv Peth
Narayan Peth
Vidyanagar (Pune)
Airport (Pune)
Chinchwadgaon
Thergaon
Punawale
Kasarwadi
Akurdi
Bibvewadi
Market Yard (Pune)
Bhusari Colony
Kothrud
Wanowarie
AFMC
Vadgaon Budruk
Bhavani Peth
Guruwar Peth
Swargate
Ghorpade Peth
N I A
Katraj
Kondhwa BK
Kondhwa Lh
Anandnagar (Pune)
Navsahyadri
Wakad
Warje


In [27]:
print(new_Pune_data_venues.shape)
new_Pune_data_venues.head()

(516, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Pune,18.521428,73.854454,Lal Mahal,18.518720,73.856556,Historic Site
1,Pune,18.521428,73.854454,Shaniwar Wada,18.520151,73.855187,Historic Site
2,Pune,18.521428,73.854454,Sai Service Bajaj,18.522260,73.850185,Motorcycle Shop
3,Pune,18.521428,73.854454,New Poona Bakery,18.517028,73.854845,Bakery
4,Pune,18.521428,73.854454,New Vaijanath Amrutatulya,18.519178,73.851471,Snack Place


In [28]:
new_Pune_data_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
AFMC,2,2,2,2,2,2
Airport (Pune),3,3,3,3,3,3
Akurdi,15,15,15,15,15,15
Anandnagar (Pune),16,16,16,16,16,16
Armament,3,3,3,3,3,3
Aundh T S,29,29,29,29,29,29
Bajirao Road,5,5,5,5,5,5
Baner Road,8,8,8,8,8,8
Bhavani Peth,7,7,7,7,7,7


In [29]:
print('There are {} uniques categories.'.format(len(new_Pune_data_venues['Venue Category'].unique())))

There are 96 uniques categories.


### 6. Analyzing each neighborhood

In [30]:
new_Pune_data_onehot = pd.get_dummies(new_Pune_data_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
new_Pune_data_onehot['Neighborhood'] = new_Pune_data_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [new_Pune_data_onehot.columns[-1]] + list(new_Pune_data_onehot.columns[:-1])
new_Pune_data_onehot = new_Pune_data_onehot[fixed_columns]

new_Pune_data_onehot.head()

,Neighborhood,ATM,Accessories Store,Airport Service,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bar,Bistro,Breakfast Spot,Burger Joint,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Concert Hall,Convenience Store,Cricket Ground,Department Store,Dessert Shop,Diner,Dumpling Restaurant,Electronics Store,Exhibit,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food Truck,Frozen Yogurt Shop,Fruit & Vegetable Store,Garden,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hockey Field,Hookah Bar,Hospital,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Italian Restaurant,Jewelry Store,Juice Bar,Kerala Restaurant,Kids Store,Korean Restaurant,Lounge,Maharashtrian Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Multiplex,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Plaza,Pub,Racetrack,Restaurant,Road,Sandwich Place,Seafood Restaurant,Shopping Mall,Snack Place,South Indian Restaurant,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Tea Room,Tennis Court,Theater,Theme Park,Track Stadium,Trail,Train,Train Station,Vegetarian / Vegan Restaurant,Watch Shop,Whisky Bar,Women's Store,Zoo
0,Pune,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Pune,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Pune,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Pune,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Pune,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [31]:
new_Pune_data_onehot.shape

(516, 97)

In [32]:
new_Pune_grouped = new_Pune_data_onehot.groupby('Neighborhood').mean().reset_index()
new_Pune_grouped

,Neighborhood,ATM,Accessories Store,Airport Service,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bar,Bistro,Breakfast Spot,Burger Joint,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Concert Hall,Convenience Store,Cricket Ground,Department Store,Dessert Shop,Diner,Dumpling Restaurant,Electronics Store,Exhibit,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food Truck,Frozen Yogurt Shop,Fruit & Vegetable Store,Garden,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hockey Field,Hookah Bar,Hospital,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Italian Restaurant,Jewelry Store,Juice Bar,Kerala Restaurant,Kids Store,Korean Restaurant,Lounge,Maharashtrian Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Multiplex,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Plaza,Pub,Racetrack,Restaurant,Road,Sandwich Place,Seafood Restaurant,Shopping Mall,Snack Place,South Indian Restaurant,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Tea Room,Tennis Court,Theater,Theme Park,Track Stadium,Trail,Train,Train Station,Vegetarian / Vegan Restaurant,Watch Shop,Whisky Bar,Women's Store,Zoo
0,AFMC,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
1,Airport (Pune),0.000000,0.0000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
2,Akurdi,0.000000,0.0000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.066667,0.0000,0.000000,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.066667,0.000000,0.000000,0.066667,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.066667,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.0,0.0,0.066667,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
3,Anandnagar (Pune),0.000000,0.0625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0625,0.000000,0.125000,0.062500,0.000000,0.125000,0.000000,0.000000,0.0,0.000000,0.062500,0.000000,0.000000,0.000000,0.0

In [33]:
new_Pune_grouped.shape

(62, 97)

### 7.Data relevant to our business problem

In [34]:
len(new_Pune_grouped[new_Pune_grouped["Bakery"] > 0])

21

In [35]:
Pune_Bakery = new_Pune_grouped[['Neighborhood','Bakery']]

In [36]:
Pune_Bakery.rename(columns = {'Neighborhood':'Post Office'},inplace = True)

C:\Users\Ninad\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [37]:
Pune_Bakery

,Post Office,Bakery
0,AFMC,0.000000
1,Airport (Pune),0.000000
2,Akurdi,0.066667
3,Anandnagar (Pune),0.000000
4,Armament,0.000000
5,Aundh T S,0.000000
6,Bajirao Road,0.200000
7,Baner Road,0.000000
8,Bhavani Peth,0.142857
9,Bhosari I E,0.000000


### 8.Clustering

In [38]:
# set number of clusters
kclusters = 4

PB_clustering = Pune_Bakery.drop('Post Office', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(PB_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 3, 1, 1, 1, 2, 1, 0, 1])

In [39]:
# add clustering labels
Pune_Bakery.insert(0, 'Cluster Labels', kmeans.labels_)



In [40]:
Pune_Bakery.head()

,Cluster Labels,Post Office,Bakery
0,1,AFMC,0.000000
1,1,Airport (Pune),0.000000
2,3,Akurdi,0.066667
3,1,Anandnagar (Pune),0.000000
4,1,Armament,0.000000


In [41]:
cols = ['Post Office','Bakery','Cluster Labels']
Pune_Bakery = Pune_Bakery[cols]
Pune_Bakery.head()

,Post Office,Bakery,Cluster Labels
0,AFMC,0.000000,1
1,Airport (Pune),0.000000,1
2,Akurdi,0.066667,3
3,Anandnagar (Pune),0.000000,1
4,Armament,0.000000,1


In [42]:
Pune_Bakery = Pune_Bakery.join(new_Pune_data.set_index("Post Office"), on="Post Office")

print(Pune_Bakery.shape)
Pune_Bakery.head() # check the last columns!

(62, 6)


,Post Office,Bakery,Cluster Labels,Pin Code,Latitudes,Longitudes
0,AFMC,0.000000,1,411040,18.5039,73.8901
1,Airport (Pune),0.000000,1,411032,18.5804,73.9184
2,Akurdi,0.066667,3,411035,18.6486,73.7647
3,Anandnagar (Pune),0.000000,1,411051,18.5083,73.8152
4,Armament,0.000000,1,411021,18.5378,73.7963


In [43]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Pune_Bakery['Latitudes'], Pune_Bakery['Longitudes'], Pune_Bakery['Post Office'], Pune_Bakery['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 0

In [48]:
Pune_Bakery.loc[Pune_Bakery['Cluster Labels'] == 0]

,Post Office,Bakery,Cluster Labels,Pin Code,Latitudes,Longitudes
8,Bhavani Peth,0.142857,0,411042,18.5214,73.8545
19,Film Institute,0.142857,0,411004,18.5214,73.8545
35,Market Yard (Pune),0.142857,0,411037,18.5214,73.8545
37,N I A,0.142857,0,411045,18.5214,73.8545
38,Nana Peth,0.125000,0,411002,18.5184,73.8563
44,Pune,0.142857,0,411001,18.5214,73.8545
45,Pune City,0.142857,0,411002,18.5214,73.8545
57,Vidyanagar (Pune),0.142857,0,411032,18.5214,73.8545


### Cluster 1

In [49]:
Pune_Bakery.loc[Pune_Bakery['Cluster Labels'] == 1]

,Post Office,Bakery,Cluster Labels,Pin Code,Latitudes,Longitudes
0,AFMC,0.000000,1,411040,18.5039,73.8901
1,Airport (Pune),0.000000,1,411032,18.5804,73.9184
3,Anandnagar (Pune),0.000000,1,411051,18.5083,73.8152
4,Armament,0.000000,1,411021,18.5378,73.7963
5,Aundh T S,0.000000,1,411007,18.5619,73.8102
7,Baner Road,0.000000,1,411008,18.5646,73.7751
9,Bhosari I E,0.000000,1,411026,18.621,73.8501
10,Bhusari Colony,0.000000,1,411038,18.5104,73.7946
12,Botanical Garden (Pune),0.000000,1,411020,18.5656,73.825
13,Chinchwad East,0.000000,1,411019,18.6404,73.7917


### Cluster 2

In [50]:
Pune_Bakery.loc[Pune_Bakery['Cluster Labels'] == 2]

,Post Office,Bakery,Cluster Labels,Pin Code,Latitudes,Longitudes
6,Bajirao Road,0.20,2,411002,18.5212,73.8552
11,Bibvewadi,0.20,2,411037,18.4782,73.8621
18,East Khadki,0.25,2,411003,18.5682,73.8508
28,Khadki,0.25,2,411003,18.5682,73.8508
60,Wanowarie,0.25,2,411040,18.4848,73.9049


### Cluster 3

In [51]:
Pune_Bakery.loc[Pune_Bakery['Cluster Labels'] == 3]

,Post Office,Bakery,Cluster Labels,Pin Code,Latitudes,Longitudes
2,Akurdi,0.066667,3,411035,18.6486,73.7647
30,Kondhwa BK,0.090909,3,411048,18.4777,73.8941
31,Kondhwa Lh,0.090909,3,411048,18.4777,73.8941
39,Narayan Peth,0.047619,3,411030,18.5156,73.8511
40,Navsahyadri,0.066667,3,411052,18.4956,73.8282
49,Shaniwar Peth (Pune),0.083333,3,411030,18.5193,73.8525


### 9.Conclusion

Bakeries are saturated in the central and southern part of the city as obserevd in clusters 2,3 and 0. The areas in cluster 1 have little to no competition. If one desires to open a new bakery or expand their franchise , the upcoming residential areas are the best since they are growing at a rapid pace but competition is very less.The best residential area can be further analysed by exploring the areas in cluster 1 even further on the basis of supermarkets,industries,property prices etc in order to find the best place to start the business  